In [28]:
from last_stretch import shortest_path
from shortest_path import filter_categories, get_position
from init import init
from locations import locations, categories
from if_else_branch import if_else_branch
from location_data import location_data
import re
file = open("auto_teleport_loop.gpc", "w")
dr = "SWI_RIGHT"
dl = "SWI_LEFT"
du = "SWI_UP"
dd = "SWI_DOWN"
plus = "SWI_PLUS"
min = "SWI_MINUS"
ab = "SWI_A"
yb = "SWI_Y"
bb = "SWI_B"
xb = "SWI_X"
zr = "SWI_ZR"
zl = "SWI_ZL"
rb = "SWI_R"
lb = "SWI_L"

tab = '\t'

# Arguments: controller input
get_val = "get_val"

# Arguments: input, intensity out of 100
set_val = "set_val"

# Argument: Waittime in milliseconds i.e. 1000ms = 1s
wait = "wait"

# Argument: combo to stop
combo_stop = "combo_stop"

# Argument: combo to run
combo_run = "combo_run"

# Arguments 1 selection of LED_1 through LED_4 and a boolean on whether it is on or off
# Update when colors known
set_led = "set_led"

# DISPLAY to screen with print(x_coordinates, y_coordinates, font_size, color, string_variable)
prnt = "print"

# Arguments: controller_input
# Different from get_val because it prevents the game from reading it in as a normal input
event_press = "event_press"

locations_per_catagory = [18, 9, 17, 20]
wait_time = [100]

In [29]:
gpc_location_array_by_type = [
    ["Set Location:\nCategory: UP/DOWN\nLocation: Left/Right"],
    ["Facilities",
    "Centrico Plaza", "Gare de Lumiose", "Pokemon Research Lab", "Hotel Z", "Racine Construction", "Restaurant Le Nah", "Rust Syndicate Office",
    "Lumiose Sewers (Main Access)", "Quasartico Inc.", "Lysandre Cafe", "Lumiose Sewers (Canal Access)", "Hotel Richissime", "Looker Bureau", "Lumiose Museum",
    "Restaurant Le Yeah", "Sushi High Roller", "Restaurant Le Wow", "Justice Dojo"],
    ["Pokemon Centers",
    "Vert Pokemon Center", "Bleu Pokemon Center", "Vernal Pokemon Center", "Magenta Pokemon Center", "Magenta Plaza Pokemon Center", "Rouge Pokemon Center", "Centrico Pokemon Center",
    "Jaune Pokemon Center", "Hibernal Pokemon Center"],
    ["Cafes",
    "Cafe Cyclone", "Cafe Classe", "Cafe Introversion", "Nouveau Cafe", "Cafe Woof", "Cafe Soleil", "Shutterbug Cafe",
    "Nouveau Cafe Truck No. 2", "Cafe Pokemon-Amie", "Cafe Rouleau", "Cafe Gallant", "Cafe Triste", "Nouveau Cafe Truck No. 3", "Cafe Ultimo",
    "Cafe Action", "Cafe Kizuna", "Cafe Bataille"],
    ["Wild Zones",
    "Wild Zone 1", "Wild Zone 2", "Wild Zone 3", "Wild Zone 4", "Wild Zone 5", "Wild Zone 6", "Wild Zone 7",
    "Wild Zone 8", "Wild Zone 9", "Wild Zone 10", "Wild Zone 11", "Wild Zone 12", "Wild Zone 13", "Wild Zone 14",
    "Wild Zone 15", "Wild Zone 16", "Wild Zone 17", "Wild Zone 18", "Wild Zone 19", "Wild Zone 20"]
]

In [30]:
commands = {"get_val: gets an input without disrupting normal game inputs",
            "set_val: inputs a selected button with a percentage argument for how far down the button press is, note: buttons also need unpressed when pressed with this function",
            "wait: waits based on an input value measured in milliseconds",
            "combo_run: invokes a function or 'combo' to run with a combo name as an argument",
            "event_press: gets an input from controller while obstructing the input from being read in by the game"}


In [31]:
def write_command(command_name, input_array, tab_num = 1):
    file.write(f'{tab * tab_num + command_name}(')
    for input in input_array[:-1]:
        file.write(f'{input}, ')
    file.write(f'{input_array[-1]});\n')
        

In [32]:
# Good for combos, if/elses, for, while
def function_firstline(function_name, function_type = 'combo', num_tabs = 0, num_spaces = 1):
    file.write(f'{tab*num_tabs}{function_type}{' '*num_spaces}{function_name}' + r' {' + '\n')

In [33]:
def end_combo(tab_num = 0):
    file.write(f'{tab * tab_num}' + '}\n\n')

In [34]:
def cluster_commands(command_array, array_of_array_input):
    for i, command in enumerate(command_array):
        write_command(command, array_of_array_input[i])

In [35]:
def button_input(button):
    cluster_commands([set_val, wait, set_val, wait], [[button, 100], wait_time, [button, 0], wait_time])

In [36]:
def button_sequence(button_array):
    for button in button_array:
        button_input(button)

In [37]:
def string_to_variable(target_string):
    variable_name = ''
    for char in target_string.lower():
        if char in ' -':
            variable_name += '_'
        elif char in '().': continue
        else:
            variable_name += char
    return variable_name

In [38]:
def convert_array_in_string(input_array):
    array = []
    i = 0
    ph_string = ''
    check = False
    for char in input_array:
        if re.match('[-0-9]', char):
            ph_string += char
            check = True
        elif check == True and re.match('[^-0-9]', input_array):
            array.append(ph_string)
            ph_string = ''
            check = False
        else:
            continue
    return array

In [39]:
def coordinates_to_dpad(coordinates):

    x_coordinate_button = dr
    y_coordinate_button = dd
    dpad_array = []

    if coordinates[0] < 0: x_coordinate_button = dl

    if coordinates[1] < 0: y_coordinate_button = du
    
    if coordinates[0] != 0:
        for i in range(abs(coordinates[0])):
            dpad_array.append(x_coordinate_button)

    if coordinates[1] != 0:
        for i in range(abs(coordinates[1])):
            dpad_array.append(y_coordinate_button)
    
    dpad_array.append(ab)
    
    # print(dpad_array)
    # print('\n\n')
    return dpad_array

In [40]:
def map_to_menu():
    function_firstline("map_to_menu")
    button_sequence([plus, yb, min])
    end_combo()

In [41]:
def menu_selection(category):
    category_index = get_position(category, filter_categories)
    category = string_to_variable(category)
    function_firstline(f'{category}_map_to_menu')
    write_command(combo_run, ["map_to_menu"])
    dpad_inputs = []
    for i in range(category_index):
        dpad_inputs.append(dd)
    dpad_inputs.append(ab)
    button_sequence(dpad_inputs)
    end_combo()

In [42]:
def create_reused_combos():
    map_to_menu()
    for category in filter_categories[:-1]:
        menu_selection(category)

In [43]:
def gpc_locations_by_type(name, category, coordinates):
    category = string_to_variable(category)
    variable_name = string_to_variable(name)
    function_firstline(f'fast_travel_{variable_name}')
    write_command("combo_run", [f'{category}_map_to_menu'])
    button_presses = coordinates_to_dpad(coordinates)
    button_sequence(button_presses)
    write_command(wait, [1000], 1)
    end_combo()

In [44]:
def generate_if_else():
    i = 1
    while i in range(len(gpc_location_array_by_type)):
        num_tabs = 3
        function_firstline(f'(category == {i})', 'if', num_tabs)
        j = 1
        while j in range(len(gpc_location_array_by_type[i])):
            num_tabs = 4
            function_name = gpc_location_array_by_type[i][j]
            function_name = string_to_variable(function_name)
            function_firstline(f'(location == {j})', 'if', num_tabs, 0)
            num_tabs = 5
            write_command('combo_run', [f'fast_travel_{function_name}'], num_tabs)
            end_combo(4)
            j += 1
        end_combo(3)
        i += 1

In [45]:
def generate_main():
    function_firstline('main', '', num_spaces=0)
    file.write(if_else_branch)
    generate_if_else()
    end_combo(2)
    end_combo(1)
    end_combo()

In [46]:
def insert_coordinate_data(array):
    file.write('const int coordinates[][] = ' + '{\n')
    for item in array[:-1]:
        file.write('{'+ f'{item[0]}, {item[1]}' + '}, ')
    file.write( r'{'+ f'{array[-1][0]}, {array[-1][1]}' + r'}' + '\n};\n\n')

In [47]:
def insert_string_array_data(location_data):
    variable_names = ["locations", "categories"]
    i = 0

    for array in location_data:
        file.write(f'const string {variable_names[i]}[][] = ' + '{\n')
        for item in array[:-1]:
            file.write(r'"'+ f'{item}' + r'", ')
        file.write(r'"' + f'{array[-1]}' + r'"'+ "\n};\n\n")
        i += 1



In [48]:
def insert_location_data(location_data=location_data):
    string_arrays = location_data[:-1]
    coordinate_array = location_data[-1]
    insert_string_array_data(string_arrays)
    insert_coordinate_data(coordinate_array)


In [49]:
def declare_variables():
    file.write('int SCRIPT_RUNNING = FALSE;\n')
    file.write('int category = 0;\n')
    file.write('int location = 0;\n')
    file.write("const int locations_per_catagory[] = {18, 9, 17, 20};\n")

In [50]:
def init_variables():
    declare_variables()
    insert_location_data()

In [51]:
def main():
    init_variables()
    file.write(init)
    create_reused_combos()


In [52]:
# for location in locations[:]:
#     coordinates, name, category  = shortest_path(location)
#     gpc_locations_by_type(name, category, coordinates)
# generate_main()

In [53]:
main()

In [54]:
# array_for_export = [[], [], []]
# for location in locations[:]:
#     coordinates, name, category  = shortest_path(location)
#     array_for_export[0].append(name)
#     array_for_export[1].append(category)
#     array_for_export[2].append(coordinates)
# with open("location_data.py", "w") as f:
#     f.write(f'''location_data = [\n''')
#     for array in array_for_export[:-1]:
#         f.write(f'{array},\n')
#     f.write(f'{array_for_export[-1]}\n]')